In [59]:
# %load test_login.py
import random
import json
import requests
from time import sleep, time
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from custom_conditions import element_has_css_class
from untils import find_report_link, watch_videos
from solve_recaptcha import write_stat, check_exists_by_xpath, wait_between, dimention, solve_images

geckodriver = 'D:\\Code\\report-yt-backend\\etc\\geckodriver-v0.21.0-win64\\geckodriver.exe'
binary = 'C:\\Program Files\\Mozilla Firefox\\firefox.exe'
PROXY_HOST = ''
PROXY_PORT = ''
email = 'cassiead16@gmail.com'
password = 'Minh12345'
recovery_mail = '1satisfyingbreath09693@yahoo.com'
report_channel = 'https://www.youtube.com/channel/UCbeWyhKo7Dk0Znf_ik0tsHw'
report_link = find_report_link(report_channel)
print(report_link)
report_reson_1 = 'Hate Speech Against a Protected Group'
report_reson_2 = 'Race or ethnic origin (examples: African American, Pacific Islander)'
report_note = 'none'
capabilities = DesiredCapabilities.FIREFOX.copy()
capabilities['marionette'] = True
capabilities['acceptSslCerts'] = True

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.privatebrowsing.autostart", True)
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False) 
profile.set_preference("browser.download.folderList", 2);
profile.set_preference("browser.download.dir", 'D:\\Code\\repport-yt-backend\\audio');
profile.set_preference("browser.download.manager.showWhenStarting", True);
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "audio/mp3");
#profile.set_preference("network.proxy.type", 1)
#profile.set_preference("network.proxy.http",PROXY_HOST)
#profile.set_preference("network.proxy.http_port",int(PROXY_PORT))
#profile.set_preference("general.useragent.override","whater_useragent")
options = webdriver.FirefoxOptions()
#options.add_argument('-headless')


browser = webdriver.Firefox(
        executable_path=geckodriver,
        firefox_options=options,
        capabilities=capabilities,
        firefox_binary=binary,
        firefox_profile=profile)
browser.set_window_size(1920, 1080)
browser.set_window_position(0, 0)
wait = ui.WebDriverWait(browser,10)



https://www.youtube.com/reportabuse?u=beWyhKo7Dk0Znf_ik0tsHw


# Login

In [47]:
from selenium.webdriver.common.action_chains import ActionChains
browser.get('https://www.youtube.com/')

sign_in_btn = browser.find_element_by_css_selector('a.yt-simple-endpoint.style-scope.ytd-button-renderer')
sign_in_btn.click()

email_inp = browser.find_element_by_id('identifierId')
email_inp.send_keys(email)
browser.find_element_by_id('identifierNext').click() # đi đến trang nhập password
wait.until(expected_conditions.text_to_be_present_in_element((
                By.CSS_SELECTOR, '#headingText > content'), 'Welcome'))
password_inp = browser.find_element_by_css_selector('input.whsOnd.zHQkBf')
password_inp.send_keys(password)
browser.find_element_by_id('passwordNext').click()

try:
    header = browser.find_element_by_id('headingText')
    if header and header.text == 'Verify it\'s you':
        print('Need to input recovery email')
        browser.find_element_by_class_name('vdE7Oc').click()
        sleep(5)
        recovery_email = browser.find_element_by_css_selector('#knowledge-preregistered-email-response')
        recovery_email.send_keys(recovery_mail)
        next_btn = browser.find_element_by_id('next')
        next_btn.click()
except Exception as ex:
    print(ex)
    print('No need to input recovery email')

wait.until(expected_conditions.url_matches('https://www.youtube.com/'))

Message: Unable to locate element: [id="headingText"]

No need to input recovery email


True

# Submit report

In [49]:
# watch some videos
browser.get(report_channel + '/videos')
videos = browser.find_elements_by_id('video-title')
video = random.choice(videos)
href = video.get_attribute('href')
watch_videos(browser, href)
        
sleep(random.randint(0, 100))
# Report this channel
browser.get(report_link)
tab_start = browser.find_element_by_id('tab-start')
report_types = tab_start.find_elements_by_css_selector('ul > li > div > label')
for report_type in report_types:
    if report_type.text == report_reson_1:
        report_type.click()

tab_hate_speech = browser.find_element_by_id('tab-hate-speech')
report_types = tab_hate_speech.find_elements_by_css_selector('ul > li > div > label')
for report_type in report_types:
    if report_type.text == report_reson_2:
        report_type.click()
        
continue_btn = browser.find_element_by_id('show-reported-user-info')
continue_btn.click()

wait.until(element_has_css_class((By.TAG_NAME, 'textarea'), "yt-uix-form-input-textarea"))
notes = browser.find_element_by_class_name('yt-uix-form-input-textarea')
notes.send_keys(report_note)

html = browser.find_element_by_tag_name('html')
html.send_keys(Keys.END)

# Goto recaptcha

## get src using in 2captcha api

In [50]:
def get_key_recaptcha():
    browser.switch_to.default_content()
    iframeSwitch = browser.find_element(By.XPATH, "/html/body/div[1]/div[3]/div/div/form/div[2]/div/div[19]/div[4]/div/div/div/iframe")
    key = iframeSwitch.get_attribute('src')
    keys = key.split('&')
    for item in keys:
        if 'k=' in item:
            key = item[2:]
    return key

In [51]:
def key_resolver_captcha(api_key, api_url):
    response_key = None
    request_id = None
    r = requests.get(api_url)
    res = r.text
    if 'OK' in res:
        request_id = res[3:]
        resolver_api = 'http://2captcha.com/res.php?key={}&action=get&id={}'.format(api_key, request_id)
        print(resolver_api)
        is_resolver = False
        while not is_resolver:
            response = requests.get(resolver_api)
            response = response.text
            if 'OK' in response:
                is_resolver = True
                response_key = response[3:]
                break
            sleep(5)
        return response_key
    else:
        print('Can not get key api 2catcha.com')

## get key

In [52]:
api_key = '094c2420f179731334edccbf176dbd79'
googlekey = get_key_recaptcha()
current_url = browser.current_url
captcharesolver_api = 'http://2captcha.com/in.php?key={}&method=userrecaptcha&googlekey={}&pageurl={}&here=now'.format(api_key, googlekey, current_url)
key_resolver = key_resolver_captcha(api_key, captcharesolver_api)

http://2captcha.com/res.php?key=094c2420f179731334edccbf176dbd79&action=get&id=60368270165


## resolver captcha after receiver request

In [53]:
# browser.switch_to.default_content()
# browser.delete_all_cookies()
# browser.switch_to.frame(iframeSwitch)
#ActionChains(driver).move_to_element(iframeSwitch).perform()
# browser.delete_all_cookies()
# WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "recaptcha-anchor")))
# ele = browser.find_element(By.ID, "recaptcha-anchor")
#ActionChains(driver).move_to_element(ele).perform()
# ele.click()


In [54]:
browser.switch_to.default_content()
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "g-recaptcha-response")))
browser.execute_script("document.getElementById('g-recaptcha-response').style.display = 'block';")
textarea_box = browser.find_element_by_id('g-recaptcha-response')
textarea_box.send_keys(key_resolver)

In [55]:
submit_report = browser.find_element_by_id('submit-report')
submit_report.click()

In [56]:
WebDriverWait(browser, 30).until(EC.presence_of_element_located((By.ID, "content")))
section = browser.find_element_by_css_selector('.section > p:nth-child(1)')
if section.text == 'Thank You.':
    sleep(5)
    browser.quit()

# check if channel die

In [57]:
# browser.get('https://www.youtube.com/channel/UCBZENPL84M_zY8-3LDmgsTQ/videos?view=0&flow=grid&sort=p')
browser.get('https://www.youtube.com/channel/UCOV-vYeBrgzxVmWzscPRxYQ')


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=57957): Max retries exceeded with url: /session/19e2f131-b2d6-4a16-8999-4187b6cd51a3/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001FF6F5BA048>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it',))

In [ ]:
text = browser.find_element_by_css_selector('#container > yt-formatted-string')
if text.text == 'This account has been terminated for violating Google\'s Terms of Service.':
    print('True')
